In [1]:
%pylab inline
import pandas as pd
pd.options.display.max_rows = 200
np.set_printoptions(precision=3)
pd.options.display.float_format = '{:,.3f}'.format

Populating the interactive namespace from numpy and matplotlib


In [2]:
import azure.cosmos
from pymongo import MongoClient
import json

with open('../local.settings.json') as settings_file:
    settings = json.load(settings_file)
    MONGO_URL = settings['Values']['mongo_url']
    MONGO_USERNAME = settings['Values']['mongo_username']
    MONGO_PASSWORD = settings['Values']['mongo_password']


def insert(element):
    db = get_db_connection()
    if get(element['date']) is None:
        db.insert(element)


def get_all():
    db = get_db_connection()
    return list(db.find())


def get(date):
    db = get_db_connection()
    return db.find_one({'date': date})


def get_db_connection():
    client = MongoClient(MONGO_URL)
    db = client.xkom_bot
    db.authenticate(name=MONGO_USERNAME, password=MONGO_PASSWORD)

    return db.xkom_bot_data

In [3]:
data = pd.DataFrame(get_all())
del data['_id']

In [4]:
# Check last added values
data.sort_values(by='date', ascending=False).head(5)

,Category,New price,Original price,Product name,date
2629,Podstawki chłodzące,"49,00","99,00",Podstawka chłodząca pod laptop Targus Single F...,2020-01-15 21
2628,Dyski SSD,"199,00","249,00","Dysk SSD Plextor 512GB 2,5"" SATA M8VC",2020-01-15 18
2627,Dyski SSD,"199,00","249,00","Dysk SSD Plextor 512GB 2,5"" SATA M8VC",2020-01-15 09
2626,Myszki przewodowe,"79,00","109,00",Dream Machines DM1 Pro (5000dpi),2020-01-14 18
2624,Dyski zewnętrzne i przenośne,"189,00","249,00",Dysk zewnetrzny Silicon Power Armor A30 1TB US...,2020-01-13 22


In [5]:
data['Original price'] = data['Original price'].apply(lambda x: pd.to_numeric(str(x)[:-3].replace(" ", "")))
data['New price'] = data['New price'].apply(lambda x: pd.to_numeric(str(x)[:-3].replace(" ", "")))
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)
data = data.sort_values(by='date')
year_index = data.index.year
month_index = data.index.month
dayofweek_index = data.index.dayofweek
hour_index = data.index.hour

In [6]:
print(data.dtypes)
data.head(5)

Category          object
New price          int64
Original price     int64
Product name      object
dtype: object


,Category,New price,Original price,Product name
date,,,,
2016-05-27 22:00:00,Drukarki atramentowe,389,449,Drukarka atramentowa Epson L130
2016-05-28 10:00:00,Karty graficzne,1459,1749,Gigabyte GeForce GTX970 Gaming G1 + klawiatura...
2016-05-28 22:00:00,Routery,279,449,Router Linksys EA6700 (1750Mb/s) DualBand USB
2016-05-29 10:00:00,"Notebooki / Laptopy 17,3""",4499,4749,MSI GE72 Apache i7-6700HQ/8GB/1TB/Win10X GTX960M
2016-05-29 22:00:00,Zasilacze awaryjne (UPS),379,419,Zasilacz awaryjny (UPS) APC Back-UPS CS (350VA


In [7]:
unique_categories = data['Category'].unique()
print(unique_categories)
print(data.shape)

['Drukarki atramentowe' 'Karty graficzne' 'Routery'
 'Notebooki / Laptopy 17,3"' 'Zasilacze awaryjne (UPS)'
 'Smartfony i telefony' 'Dyski SSD' 'Karty graficzne NVIDIA'
 'Notebooki / Laptopy 13,3"' 'Obudowy do komputera'
 'Zasilacze do komputera' 'Tablety 10"' 'Nagrywarki DVD' 'Brak'
 'Pendrive (pamięci USB)' 'Myszki bezprzewodowe' 'Karty pamięci microSD'
 'Uchwyty do smartfonów' 'Notebooki / Laptopy 15,6"'
 'Urządzenia wiel. atramentowe' 'Odtwarzacze MP3' 'Zestawy słuchawkowe'
 'Access Pointy' 'Wkłady do aparatów' 'Kamery sportowe'
 'Klawiatury  przewodowe' 'Etui na tablety' 'Plecaki na laptopy'
 'Głośniki komputerowe' 'Czytniki kart do smartfonów' 'Ładowarki do kamer'
 'Słuchawki bezprzewodowe' 'Słuchawki przewodowe' 'Przejściówki'
 'Powerbanki' 'Papier do drukarek' 'Akumulatory uniwersalne' 'Konsole'
 'Tablety 8"' 'Smartwatche' 'Monitory LED 24" (23,5"-26,4")'
 'Monitory LED 27" (26,5"-28,4")' 'Torby na laptopy' 'Głośniki'
 'Wideorejestratory' 'Skanery' 'Nawigacje samochodowe'
 'Ład

In [8]:
categories = ['Procesory', 'Monitory', 'Telewizory', 'Kable', 'Ładowarki', 'Stacje dokujące', 'Tablety', 'Papier', 'Gry', 'Akcesoria', 'Gaming', 'Etui', 'Laptopy', 'Słuchawk', 'Drukarki', 'Konsol', 'Karty graficzne', 'Klawiatury', 'Kamery', 'Płyty główne', 'Obudowy', 'Głośniki', 'Myszki', 'Dyski', 'Karty pamięci']
new_categories = ['Procesory', 'Monitory', 'Telewizory', 'Kable', 'Ładowarki', 'Stacje dokujące', 'Tablety', 'Papier', 'Gry', 'Akcesoria', 'Gaming', 'Etui', 'Laptopy', 'Słuchawki', 'Drukarki', 'Konsole', 'Karty graficzne', 'Klawiatury', 'Kamery', 'Płyty główne', 'Obudowy', 'Głośniki', 'Myszki', 'Dyski', 'Karty pamięci']
    
for category_name, new_name in zip(categories, new_categories):
    filtered = list(filter(lambda category: category_name.lower() in category.lower(), unique_categories))
    for name in filtered:
        data['Category'][data['Category'] == name] = new_name

/home/stachu/anaconda3/envs/master/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [9]:
most_common_categories = data['Category'].value_counts()[:5].index.tolist()
data['Category'].value_counts()[:5]

Smartfony i telefony    379
Słuchawki               216
Dyski                   175
Monitory                167
Laptopy                 154
Name: Category, dtype: int64

In [10]:
data['Percentage discount'] = (1 - data['New price']/data['Original price']) * 100
data['Percentage discount'] = data['Percentage discount'].apply(lambda value: int(value))
percentage_discount = data[['Percentage discount']]
print("%.2f" % percentage_discount.mean(), "%.2f" % percentage_discount.std())                      

24.74 14.37


In [19]:
# percentage_discount.iloc[-20:].plot(figsize=(20,10), linewidth=5, fontsize=20)
def show_basic_stats(df, index, value, name):
    print("{} {}:".format(name, value), "Mean: %.2f" % df.iloc[index==value].mean(), "Standard deviation: %.2f" % df.iloc[index==value].std()) 
for hour in [10, 22]:
    show_basic_stats(percentage_discount, hour_index, hour, "Hour")
for day in np.arange(0,7):
    show_basic_stats(percentage_discount, dayofweek_index, day, "Day")
for month in np.arange(1,13):
    show_basic_stats(percentage_discount, month_index, month, "Month")
for year in np.arange(2016, 2021):
    show_basic_stats(percentage_discount, year_index, year, "Year")

Hour 10: Mean: 24.90 Standard deviation: 14.75
Hour 22: Mean: 24.59 Standard deviation: 14.01
Day 0: Mean: 24.67 Standard deviation: 15.02
Day 1: Mean: 21.97 Standard deviation: 13.12
Day 2: Mean: 24.72 Standard deviation: 14.64
Day 3: Mean: 25.14 Standard deviation: 14.14
Day 4: Mean: 25.29 Standard deviation: 14.67
Day 5: Mean: 24.94 Standard deviation: 13.35
Day 6: Mean: 26.49 Standard deviation: 15.19
Month 1: Mean: 26.04 Standard deviation: 15.53
Month 2: Mean: 25.49 Standard deviation: 16.59
Month 3: Mean: 26.74 Standard deviation: 13.80
Month 4: Mean: 25.99 Standard deviation: 14.27
Month 5: Mean: 25.32 Standard deviation: 14.92
Month 6: Mean: 24.01 Standard deviation: 13.08
Month 7: Mean: 24.71 Standard deviation: 15.06
Month 8: Mean: 23.93 Standard deviation: 14.26
Month 9: Mean: 25.21 Standard deviation: 14.10
Month 10: Mean: 23.33 Standard deviation: 13.81
Month 11: Mean: 24.22 Standard deviation: 14.37
Month 12: Mean: 23.27 Standard deviation: 12.98
Year 2016: Mean: 21.08 S